# Day 4

In [1]:
from aocd import get_data
from collections import defaultdict, deque
import re
data = get_data(day=7).split('\n')

# --- Day 7: The Sum of Its Parts ---
You find yourself standing on a snow-covered coastline; apparently, you landed a little off course. The region is too hilly to see the North Pole from here, but you do spot some Elves that seem to be trying to unpack something that washed ashore. It's quite cold out, so you decide to risk creating a paradox by asking them for directions.

"Oh, are you the search party?" Somehow, you can understand whatever Elves from the year 1018 speak; you assume it's Ancient Nordic Elvish. Could the device on your wrist also be a translator? "Those clothes don't look very warm; take this." They hand you a heavy coat.

"We do need to find our way back to the North Pole, but we have higher priorities at the moment. You see, believe it or not, this box contains something that will solve all of Santa's transportation problems - at least, that's what it looks like from the pictures in the instructions." It doesn't seem like they can read whatever language it's in, but you can: "Sleigh kit. Some assembly required."

"'Sleigh'? What a wonderful name! You must help us assemble this 'sleigh' at once!" They start excitedly pulling more parts out of the box.

The instructions specify a series of steps and requirements about which steps must be finished before others can begin (your puzzle input). Each step is designated by a single letter. For example, suppose you have the following instructions:

```
Step C must be finished before step A can begin.
Step C must be finished before step F can begin.
Step A must be finished before step B can begin.
Step A must be finished before step D can begin.
Step B must be finished before step E can begin.
Step D must be finished before step E can begin.
Step F must be finished before step E can begin.
```

Visually, these requirements look like this:

```
  -->A--->B--
 /    \      \
C      -->D----->E
 \           /
  ---->F-----
```

Your first goal is to determine the order in which the steps should be completed. If more than one step is ready, choose the step which is first alphabetically. In this example, the steps would be completed as follows:

- Only C is available, and so it is done first.
- Next, both A and F are available. A is first alphabetically, so it is done next.
- Then, even though F was available earlier, steps B and D are now also available, and B is the first alphabetically of the three.
- After that, only D and F are available. E is not available because only some of its prerequisites are complete. - Therefore, D is completed next.
- F is the only choice, so it is done next.
- Finally, E is completed.
- So, in this example, the correct order is CABDFE.

In what order should the steps in your instructions be completed?

In [2]:
regex = r"Step (\w) must be finished before step (\w) can begin"
parse = lambda line: re.findall(regex, line)[0]
relations = list(map(parse, data))

outgoing = defaultdict(list)
incoming = defaultdict(list)
for start, end in relations: 
    outgoing[start].append(end)
    incoming[end].append(start)

In [15]:
with_no_incoming = list(sorted(filter(lambda k: all(k not in value for value in outgoing.values()), outgoing.keys())))
with_no_incoming

['O', 'U', 'X', 'Y']

In [11]:
from heapq import heapify, heappop, heappush

seen, result = set(), []
Q = with_no_incoming
heapify(Q)
while len(Q) > 0:
    node = heappop(Q)
    seen.add(node)
    result.append(node)
    for out in sorted(outgoing[node]):
        if all(x in seen for x in incoming[out]):
            heappush(Q, out)
''.join(result)

'OUGLTKDJVBRMIXSACWYPEQNHZF'

## --- Part Two ---
As you're about to begin construction, four of the Elves offer to help. "The sun will set soon; it'll go faster if we work together." Now, you need to account for multiple people working on steps simultaneously. If multiple steps are available, workers should still begin them in alphabetical order.

Each step takes 60 seconds plus an amount corresponding to its letter: A=1, B=2, C=3, and so on. So, step A takes 60+1=61 seconds, while step Z takes 60+26=86 seconds. No time is required between steps.

To simplify things for the example, however, suppose you only have help from one Elf (a total of two workers) and that each step takes 60 fewer seconds (so that step A takes 1 second and step Z takes 26 seconds). Then, using the same instructions as above, this is how each second would be spent:

```
Second   Worker 1   Worker 2   Done
   0        C          .        
   1        C          .        
   2        C          .        
   3        A          F       C
   4        B          F       CA
   5        B          F       CA
   6        D          F       CAB
   7        D          F       CAB
   8        D          F       CAB
   9        D          .       CABF
  10        E          .       CABFD
  11        E          .       CABFD
  12        E          .       CABFD
  13        E          .       CABFD
  14        E          .       CABFD
  15        .          .       CABFDE
```

Each row represents one second of time. The Second column identifies how many seconds have passed as of the beginning of that second. Each worker column shows the step that worker is currently doing (or . if they are idle). The Done column shows completed steps.

Note that the order of the steps has changed; this is because steps now take time to finish and multiple workers can begin multiple steps simultaneously.

In this example, it would take 15 seconds for two workers to complete these steps.

With 5 workers and the 60+ second step durations described above, how long will it take to complete all of the steps?



In [17]:
time = lambda letter: ord(letter) - ord('A') + 61

seen = set()
Q = [(0, x) for x in with_no_incoming]
heapify(Q)
while len(Q) > 0:
    cost, node = heappop(Q)
    end_cost = cost + time(node)
    print([node, cost, end_cost])
    seen.add(node)
    for out in sorted(outgoing[node]):
        if all(x in seen for x in incoming[out]):
            heappush(Q, (end_cost, out))

['O', 0, 75]
['U', 0, 81]
['X', 0, 84]
['Y', 0, 85]
['G', 81, 148]
['L', 81, 153]
['V', 81, 163]
['T', 148, 228]
['R', 163, 241]
['K', 228, 299]
['M', 241, 314]
['B', 299, 361]
['D', 299, 363]
['I', 314, 383]
['J', 363, 433]
['S', 363, 442]
['A', 442, 503]
['C', 442, 505]
['P', 503, 579]
['W', 503, 586]
['E', 579, 644]
['Q', 644, 721]
['N', 721, 795]
['Z', 721, 807]
['H', 795, 863]
['F', 863, 929]
